## Paso 0: Acceso a Datos 
Para garantizar la persistencia de los datos y simular un entorno de producción donde los datos residen en la nube, importamos los datos desde la carpeta data/.

In [ ]:
# Importación de librerías
import pandas as pd 
import os

# Configuración de rutas 
DATA_PATH = "../data/"

# Diccionario de archivos 
files = {
    "customers": "olist_customers_dataset.csv",
    "items": "olist_order_items_dataset.csv",
    "orders": "olist_orders_dataset.csv",
    "products": "olist_products_dataset.csv",
    "translation": "product_category_name_translation.csv"
}

def load_olist_data(directory, file_map):
    """Carga los datasets y verifica su existencia."""
    loaded_data = {}
    for key, name in file_map.items():
        full_path = os.path.join(directory, name)
        if os.path.exists(full_path):
            loaded_data[key] = pd.read_csv(full_path)
            print(f"{name} cargado correctamente.")
        else:
            print(f"Error: No se encuentra {name} en la ruta {directory}")
    return loaded_data

# Ejecución de la carga
datasets = load_olist_data(DATA_PATH, files)

# Asignación de variables globales para el análisis
df_customers = datasets.get("customers")
df_items = datasets.get("items")
df_orders = datasets.get("orders")


# Data Analytics & Segmentación IA - Olist E-commerce
**AUTOR:** DENISLMO
**DATASET:** Olist Brazil (Kaggle)

## Paso 1: Configuración de entorno
En esta sección improtamos las librerías necesarias y preparamos un motor **SQL (SQLite)**
Utilizamos SQL para la extracción de datos porque es más eficiente en entornos productivos que cargan archivos masivos directamente en memoria en Pandas.

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 

# Estetica gráfica
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


# 1 Carga de datos - Creamos función
def load_olist_data(file_name):
  return pd.read_csv(os.path.join(DATA_PATH, file_name))



try:
# Cargamos solo lo necesario para este hito
  orders = load_olist_data('olist_orders_dataset.csv')
  items = load_olist_data('olist_order_items_dataset.csv')
  products = load_olist_data('olist_products_dataset.csv')
  customers = load_olist_data('olist_customers_dataset.csv')

  # Esta tabla es la clave para traducir las categorías
  translation = load_olist_data('product_category_name_translation.csv')

  # 2 Conexión a motor SQL en memoria
  conn = sqlite3.connect(':memory:')

  # 3 Transferencia a SQL

  orders.to_sql('orders', conn, index=False)
  items.to_sql('items', conn, index=False)
  products.to_sql('products', conn, index=False)
  translation.to_sql('translation', conn, index=False)
  customers.to_sql('customers', conn, index=False)

  print("Datos cargados y motor SQL activo")
except Exception as e:
  print(f"Error al cargar los datos: {e}")


## Paso 2: Identificación de Categorías Estratégicas
¿Donde genera Olist su dinero? Antes de aplicar modelos de ML, necesitamos entender que productos tienen el mayor impacto financiero.

Realizamos un 'JOIN' para unir los precios de los artículos con sus nombres traducidos al inglés.

In [ ]:
inspector = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tablas disponibles")
print(inspector)

query = """
SELECT
      t.product_category_name_english AS categoria,
      SUM(i.price) AS ingresos_totales
FROM items i
JOIN products p ON i.product_id = p.product_id
JOIN translation t ON p.product_category_name = t.product_category_name
GROUP BY 1
ORDER BY ingresos_totales DESC
LIMIT 10;
"""

df_top_revenue = pd.read_sql_query(query, conn)
df_top_revenue

# Graficamos
plt.figure(figsize=(12,6))
sns.barplot(data=df_top_revenue, x='ingresos_totales', y='categoria',palette='viridis')
plt.title('Top 10 categorías por facturación', fontsize=15)
plt.show()

## Paso 4: Análisis de Lealtad y Recurrencia
**Pregunta de Negocio:** ¿Nuestros clientes compran más de una vez o somos una plataforma de "una sola compra"?
Este análisis es vital antes de entrenar un modelo de Machine Learning, ya que determina si debemos enfocarnos en **Adquisición** o en **Retención**.

In [ ]:
# Query para contar pedidos por cliente único

query_frecuencia = """
SELECT
      customer_unique_id,
      COUNT(order_id) AS n_pedidos
FROM orders o
JOIN (SELECT customer_id , customer_unique_id FROM customers) c
  ON o.customer_id = c.customer_id
GROUP BY 1
ORDER BY n_pedidos DESC;
"""

df_frecuencia = pd.read_sql_query(query_frecuencia, conn)
df_frecuencia

# Visualización
plt.figure(figsize=(10,5))
sns.countplot(data=df_frecuencia[df_frecuencia['n_pedidos']< 5], x='n_pedidos', palette='Blues')
plt.title('Distribución de pedidos por cliente', fontsize=15)
plt.xlabel('Número de pedidos', fontsize=12)
plt.ylabel('Número de clientes', fontsize=12)
plt.show()

# Mostramos el porcentaje de clientes que repiten

repiten = (df_frecuencia['n_pedidos']>1).sum()
total_clientes = len(df_frecuencia)
print(f"El {(repiten/total_clientes)*100:.2f}% de clientes han comprado más de una vez. ")


## Paso 5: Preparación de Datos para IA (Modelo RFM)
Para que un modelo de Machine Learning (como K-Means) pueda segmentar clientes, necesitamos transformar los datos transaccionales en variables numéricas. Utilizaremos la metodología **RFM**:

1. **Recency (Recencia):** ¿Hace cuántos días fue su última compra?
2. **Frequency (Frecuencia):** ¿Cuántas veces ha comprado?
3. **Monetary (Monetario):** ¿Cuánto dinero ha gastado en total?

*Nota Técnica:* Como el dataset es histórico, calcularemos la "Recencia" tomando como referencia el día posterior a la última compra registrada en toda la base de datos.

In [ ]:
# Query para crear nuestra "Feature Table" para la IA

query_rfm = """
WITH facturacion_por_pedido AS (
  SELECT order_id , SUM(price) as valor_pedido
  FROM items
  GROUP BY 1
),
datos_clientes AS(
  SELECT
    c.customer_unique_id,
    o.order_purchase_timestamp,
    f.valor_pedido
  FROM orders o
  JOIN customers c ON o.customer_id = c.customer_id
  JOIN facturacion_por_pedido f ON o.order_id = f.order_id
  WHERE o.order_status = 'delivered'
)
SELECT
  customer_unique_id,
  CAST((julianday((SELECT MAX(order_purchase_timestamp) FROM orders)) - julianday(MAX(order_purchase_timestamp))) AS INT) AS recencia,
  COUNT(*) AS frecuencia,
  SUM(valor_pedido) AS monetario
FROM datos_clientes
GROUP BY 1;
"""

df_rfm = pd.read_sql_query(query_rfm, conn)
df_rfm

# Mostramos las primeras filas
print("Tabla RFM lista para el modelo de ML:")
display(df_rfm.head())

###  Análisis de Outliers en RFM
Antes de entrenar un modelo de IA (K-Means), debemos observar la distribución de nuestros datos. Los modelos de clustering son muy sensibles a los valores atípicos (outliers), como esos clientes que gastan 100 veces más que el promedio.

In [ ]:
# Graficamos la relación entre Frecuencia y Monetario
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_rfm, x='frecuencia', y='monetario', alpha=0.5, color='teal')
plt.title('Relación Frecuencia vs Monetario (Detección de Outliers)', fontsize=15)
plt.show()

# Resumen estadístico
print(df_rfm[['recencia', 'frecuencia', 'monetario']].describe())

## Conceptos Fundamentales: El Modelo RFM

Antes de aplicar Inteligencia Artificial, debemos entender las dimensiones que definen el comportamiento de nuestros clientes. El modelo **RFM** es el estándar de la industria para segmentación:

1. **Recency (Recencia - R):** Días transcurridos desde la última compra.
   * *Lógica:* Un cliente que compró recientemente tiene la marca fresca en su mente y es más probable que responda a promociones.
2. **Frequency (Frecuencia - F):** Cantidad total de pedidos realizados.
   * *Lógica:* Mide la lealtad. Un cliente frecuente confía en la plataforma.
3. **Monetary (Monetario - M):** Valor total gastado por el cliente.
   * *Lógica:* Mide el valor económico. Ayuda a diferenciar entre clientes de alto ticket y compradores ocasionales de bajo costo.

## Paso 6: Escalado de Características (Feature Scaling)
Los algoritmos de ML basados en distancias (como **K-Means**, que usaremos para segmentar clientes) son muy sensibles a la escala de los datos.

**El Problema:**
* La **Recencia** tiene valores entre 0  y 700+
* La **Frecuencia** tiene valores mayoritariamente entre 1 y 5
* El **Monetario** tiene valores de 0 a 10.000+

Si no escalamos, el algoritmo pensará que el dinero es miles de veces más importante que
la frecuencia solo porque el número es más grande. Para solucionarlo, usaremos **StandardScaler**, que ajusta los datos para que tengan una media de 0 y una desviación estándar de 1.

In [ ]:
from sklearn.preprocessing import StandardScaler

# 1 Seleccionamos solo las columnas númericas que usaremos para la IA
features = ['recencia', 'frecuencia', 'monetario']
data_to_scale = df_rfm[features]

# 2 Inicializamos el escalador
scaler = StandardScaler()

# 3 Ajustamos y transformamos los datos
# fit_transform aprende la media y varianza y luego escala los datos
rfm_scaled = scaler.fit_transform(data_to_scale)

# 4 Convertimos de nuevo a un DataFrame para que sea más fácil de leer
df_rfm_scaled = pd.DataFrame(rfm_scaled, columns=features)
df_rfm_scaled.head()

# Añadimos el ID del cliente de nuevo para no perderlo
df_rfm_scaled['customer_unique_id'] = df_rfm.reset_index()['customer_unique_id']

print("Datos normalizados. Ahora todos los valores están en una escala similar.")
display(df_rfm_scaled.head())

###  Insight Técnico
Al aplicar `StandardScaler`, la fórmula que se ejecuta detrás es:
$$z = \frac{(x - \mu)}{\sigma}$$
Donde $x$ es el valor original, $\mu$ es el promedio y $\sigma$ es la desviación estándar.
Ahora, un "1" en frecuencia pesa lo mismo que un "1" en dinero para los ojos de nuestra IA.

##  ¿Cómo interpretar los valores normalizados?

Tras aplicar `StandardScaler`, los datos se transforman a una escala de **Z-Score** donde el **0** representa el promedio de toda la base de datos.

| Variable | Valor Positivo (+) | Valor Negativo (-) |
| :--- | :--- | :--- |
| **Recency** | Compró hace **más** días que el promedio (Cliente inactivo). | Compró hace **menos** días que el promedio (Cliente reciente). **¡Es bueno!** |
| **Frequency** | Compró **más** veces que el promedio (Leal). | Compró **menos** veces que el promedio (Ocasional). |
| **Monetary** | Gastó **más** dinero que el promedio (Alto Valor). | Gastó **menos** dinero que el promedio (Bajo Valor). |

## Paso 7: Determinación del Número de Clusters (Método del Codo)
El algoritmo **K-Means** es un aprendizaje no supervisado que agrupa puntos por cercania. Sin embargo, requiere que nosotros definamos el número de grupos (`k`).

Para no elegir un número al azar, utilizamos el **Método del Codo (Elbow Method)**:
1. Probamos el algoritmo con diferentes valores de `k` (del 1 al 10).
2. Calculamos la **Inercia** (o WCSS): la suma de las distancias al cuadrado de cada punto al centro del grupo.
3. Buscamos el punto donde la caída de la inercia se suaviza (el "codo"), lo que indica que añadir más grupos no aporta una mejora significativa.

In [ ]:
from matplotlib import lines
from sklearn.cluster import KMeans

# 1 Preparar lista para almacenar la inercia (WCSS)

wcss=[]

# 2 Bucle para probar k del 1 al 10
# Usamos random_state=42 para que los resultados sean siempre iguales
for i in range(1,11):
  kmeans = KMeans(n_clusters=i, init='k-means++' ,random_state=42, n_init=10)
  kmeans.fit(rfm_scaled)
  wcss.append(kmeans.inertia_)

# 3 Visualizamos los resultados
plt.figure(figsize=(10,6))
plt.plot(range(1,11),wcss, marker='o',color='indigo', linestyle='--')
plt.title('Método del Codo (Elbow Method)', fontsize=16, fontweight='bold')
plt.xlabel('Número de clusters (k)',fontsize=12)
plt.ylabel('WCSS',fontsize=12)
plt.xticks(range(1,11))
plt.grid(True, alpha=0.3)
plt.show()

## Paso 8: Entrenamiento del Modelo y Asignación de Clusters
Con el valor óptimo de $k=5$, procedemos a entrenar el algoritmo **K-Means**.
Este proceso agrupará a los clientes en 5 segmentos distintos basados en la proximidad de sus vectores RFM normalizados.

Posteriormente, devolveremos estas etiquetas a nuestro DataFrame original para analizar las características de cada grupo en valores reales (R$, días, unidades).

In [ ]:
# 1 Configuramos el modelo con k=5
kmeans = KMeans(n_clusters=5, init='k-means++', random_state=42, n_init=10)

# 2 Entrenamos y predecimos las etiquetas
clusters = kmeans.fit_predict(rfm_scaled)

# 3 Añadimos la etiqueta de cluster al DataFrame original (el de los datos reales)
df_rfm['cluster'] = clusters

# 4 Verificamos cuántos clientes cayeron en cada grupo
print("Cantidad de clientes por segmento:")
print(df_rfm['cluster'].value_counts())

display(df_rfm.head())

## Paso 9: Perfilamiento de los Segmentos
Para enternder qué significa cada cluster, calculamos los valores promedio de **Recencia, Frecuencia y Monetario** para cada grupo.

Esto nos permitirá ponerles "nombre y apellido" (ej. Campeones, Clientes Perdidos, etc...)

In [ ]:
# Agrupamos por cluster  y calculamos la media de las variables originales

perfil_clusters = df_rfm.groupby('cluster').agg({
    'recencia':'mean',
    'frecuencia':'mean',
    'monetario':'mean',
    'customer_unique_id':'count'
}).rename(columns={'customer_unique_id':'n_clientes'}).sort_values('monetario',ascending=False)


# Formateamos para que sea legible
pd.options.display.float_format= '{:,.2f}'.format
print("Perfil promedio de cada segmento:")
display(perfil_clusters)


## Paso 10: Definición de Estrategia por Segmento

Tras analizar los promedios, definimos las siguientes categorías para el negocio:
| Cluster | Nombre sugerido | Perfil | Estrategia de IA |
| :--- | :--- | :--- | :--- |
| **0** | **Grandes Compradores** | Gasto muy alto, pero inactivos hace meses. | Campaña de "Win-back" con productos de lujo. |
| **4** | **Clientes Fieles** | Los únicos que repiten compra (Freq > 2). | Programa de puntos y fidelización. |
| **2** | **Nuevas Promesas** | Compraron hace poco (Recencia baja). | Recomendación de productos (Cross-selling). |
| **1** | **Clientes Tibios** | Compraron hace un año, gasto bajo. | Descuentos de reactivación por tiempo limitado. |
| **3** | **Inactivos / Fugados** | No compran hace más de 500 días. | No gastar presupuesto de marketing aquí. |

## Paso 10.1: Visualización Espacial de Clientes
Para validar físicamente la segmentación de nuestras IA, proyectamos a los clientes en un espacio de tres dimensiones.

* **Eje X (Recencia):** Muestra qué tan reciente es la compra.
* **Eje Y (Frecuencia):** Muestra la lealtad.
* **Eje Z (Monetario):** Muestra el valor económico.

Esta gráfica interactiva permite identificar los "bordes" de cada cluster y entender cómo el algoritmo K-Means agrupó a los usuarios por su comportamiento similar.

In [ ]:
import plotly.express as px

# Creamos el gráfico 3D interactivo
fig = px.scatter_3d(
    df_rfm,
    x='recencia',
    y='frecuencia',
    z='monetario',
    color='cluster',        # Diferencia los grupos por color
    title='Segmentación 3D de Clientes (Modelo RFM)',
    labels={
        'recencia':'Días (R)',
        'frecuencia':'Pedidos (F)',
        'monetario':'Gasto R$ (M)'
    },
    opacity=0.6,                      # Transparencia para ver puntos solapados
    color_continuous_scale='Viridis'  # Paleta de colores pro
)

# Ajustamos el tamaño y márgenes
fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=50),
    scene=dict(
        xaxis_title='Recencia',
        yaxis_title='Frecuencia',
        zaxis_title='Monetario'
    )
)

fig.show()

## Paso 11: IA Generativa para Marketing Personalizado
En un entorno de producción moderno, no escribimos los correos a mano. Utilizamos la etiqueta del **Cluster** generada por nuestra IA de ML para alimentar un  **Prompt** que una IA Generativa (LLM) usará para crear mensajes únicos.

A continuación, creamos una función que asigna la "Instrucción de Marketing" basada en la predicción del modelo.

In [ ]:
# Definimos las instrucciones que le daríamos a una API de IA
# para cada tipo de cliente identificado por K-Means

def asignar_instruccion_ia(cluster):
  instrucciones= {
      0:"Generar un correo de lujo invitando a conocer la nueva colección premium.",
      4:"Agradecer por ser parte del 3% de clientes exclusivos y ofrecer envío gratis.",
      2:"Dar la bienvenida y sugerir productos basados en su primera categoría de compra.",
      1:"Enviar un recordatorio de 'Te extrañamos' con un 10% de descuento.",
      3:"Marcar como inactivo en el CRM para optimizar costos de envío de emails."
  }
  return instrucciones.get(cluster,"Enviar comunicación estándar.")

# Aplicamos la lógica a nuestro DataFrame
df_rfm['accion_ia'] = df_rfm['cluster'].apply(asignar_instruccion_ia)

# Visualizamos el resultado final
print("Pipeline de IA completado")
display(df_rfm[['customer_unique_id','cluster','accion_ia']].head(10))

# 🏁 RESUMEN EJECUTIVO: Olist-Analysis - Inteligencia de Clientes E-commerce

---

## 🎯 El Problema de Negocio
**Olist** presentaba una base de datos masiva (+100k registros) pero sin segmentar, con una alta dependencia de adquisición de clientes nuevos. El objetivo estratégico fue transformar estos datos transaccionales en **perfiles de comportamiento** (Customer Personas) para optimizar el presupuesto de marketing, personalizar la comunicación y aumentar la retención de usuarios.

---

## 🛠️ Stack Tecnológico
* **Gestión de Datos:** SQL (SQLite) para el cruce eficiente de tablas relacionales.
* **Procesamiento:** Python (Pandas, NumPy).
* **Machine Learning:** Scikit-Learn (StandardScaler, KMeans).
* **Visualización:** Seaborn, Plotly (3D Interactivo).
* **Estrategia:** IA Generativa (Prompt Engineering para automatización de campañas).

---

## 🗺️ Mapa de Ruta: Los 11 Hitos Logrados

| Fase | Hito | Descripción Técnica |
| :--- | :--- | :--- |
| **Ingeniería de Datos** | 1-3. SQL Joins | Conexión de `orders`, `items`, `customers` y `products` usando Aliases (`AS`). |
| **Análisis (EDA)** | 4-5. Retención | Identificación de una tasa de recurrencia crítica del **3.12%**. |
| **ML Prep** | 6. RFM | Ingeniería de características: **Recency, Frequency, Monetary**. |
| **ML Prep** | 7. Scaling | Normalización de datos con `StandardScaler` para equilibrar magnitudes. |
| **Modelado** | 8. Elbow Method | Validación de $k=5$ clusters mediante el análisis de Inercia (WCSS). |
| **Modelado** | 9. Clustering | Entrenamiento de **K-Means** y asignación de etiquetas automáticas. |
| **Estrategia** | 10. Naming | Traducción de clusters a Personas: *Champions, At Risk, New Promises*. |
| **Estrategia** | 11. AI Logic | Creación de un pipeline de comunicación personalizada vía Prompting. |

---

## 🧠 Conceptos Senior para Entrevistas

* **¿Por qué SQL y no solo Pandas?** Para simular un entorno productivo de *Data Warehouse*, garantizando la escalabilidad del análisis y optimizando el uso de memoria RAM.
* **La lógica del RFM:** * **$R$ (Recencia):** A menor valor (negativo en escala normalizada), más fresco es el cliente.
    * **$F$ (Frecuencia):** Mide la lealtad mediante el volumen de compras repetidas.
    * **$M$ (Monetario):** Indica el valor de vida del cliente (*LTV*).
* **Normalización Z-Score:** $$z = \frac{(x - \mu)}{\sigma}$$
    Donde $\mu$ es la media y $\sigma$ la desviación estándar. Este paso es crucial para que el algoritmo de IA no se sesgue por variables con números grandes (como el dinero) sobre las pequeñas (frecuencia).

---